# Random Forest and SVR with Feature Engineering

- hyperparameter optimization only done with grid search (not enough time to do a first-pass with RandomizedSearchCV or to generally do it with Optuna; and: Optuna did not bring considerably better results compared to GridSearchCV for the RF without feature engineering and no improvement at all for the SVRs without feature engineering)

In [1]:
# !conda install -c conda-forge rdkit -y

In [1]:
# import required packages
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

In [2]:
# import datasets
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
print(train.head())

id_test = test["id"] 

     id                       SMILES      Tm  Group 1  Group 2  Group 3  \
0  2175        FC1=C(F)C(F)(F)C1(F)F  213.15        0        0        0   
1  1222  c1ccc2c(c1)ccc3Nc4ccccc4c23  407.15        0        0        0   
2  2994          CCN1C(C)=Nc2ccccc12  324.15        2        1        0   
3  1704                   CC#CC(=O)O  351.15        1        0        0   
4  2526                    CCCCC(S)C  126.15        2        3        0   

   Group 4  Group 5  Group 6  Group 7  ...  Group 415  Group 416  Group 417  \
0        0        0        0        0  ...          0          0          0   
1        0        0        0        0  ...          0          0          0   
2        0        0        0        0  ...          0          0          0   
3        0        0        0        0  ...          0          0          0   
4        0        0        0        0  ...          0          0          0   

   Group 418  Group 419  Group 420  Group 421  Group 422  Group 423  Group

In [3]:
# feature engineering

from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors

def featurize_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(2050)

    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hbd = Descriptors.NumHDonors(mol)
    hba = Descriptors.NumHAcceptors(mol)
    tpsa = Descriptors.TPSA(mol)

    fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    fp = np.array(fp)

    return np.concatenate([[mw, logp, hbd, hba, tpsa], fp])

X_train = np.vstack(train["SMILES"].apply(featurize_smiles))
X_test = np.vstack(test["SMILES"].apply(featurize_smiles))
y_train = train["Tm"].values

[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerator
[17:16:13] DEPRECATION WARNING: please use MorganGenerat

In [8]:
# inspection of data with new features

print(X_train[:5])
print(X_train.shape)

# dropping columns that contain only zeros in both train and test set

zero_cols_train = np.all(X_train == 0, axis=0)
zero_cols_test = np.all(X_test == 0, axis=0)
zero_cols_both = zero_cols_train & zero_cols_test

print("Number of such columns:", zero_cols_both.sum())

X_train = X_train[:, ~zero_cols_both]
X_test = X_test[:, ~zero_cols_both]

print("New shape of X_train:", X_train.shape)

[[1.62032e+02 2.42120e+00 0.00000e+00 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 [2.17271e+02 4.47430e+00 1.00000e+00 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 [1.60220e+02 2.36462e+00 0.00000e+00 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 [8.40740e+01 9.43000e-02 1.00000e+00 ... 0.00000e+00 0.00000e+00
  0.00000e+00]
 [1.18245e+02 2.49490e+00 1.00000e+00 ... 0.00000e+00 0.00000e+00
  0.00000e+00]]
(2662, 2053)
Number of such columns: 67
New shape of X_train: (2662, 1986)


### 1.) Random Forest

In [9]:
# grid with parameters to search
parameters_grid = {
    "n_estimators": [200, 400, 600, 800],
    "max_depth": [20, 30, 40, None],
    "max_features": [0.1, 0.2, 0.3]}

# model initialisation
rf = RandomForestRegressor(min_samples_split = 2, min_samples_leaf = 1, random_state = 123)

# GridSearchCV initialisation
rf_grid_search = GridSearchCV(estimator = rf, param_grid = parameters_grid, cv = 4, scoring = "neg_mean_absolute_error", n_jobs = -1,
                              error_score = "raise")

# GridSearchCV fit to the training data
rf_grid_search.fit(X_train, y_train)

# Extraction of best result
print("Best parameters found: ", rf_grid_search.best_params_)
print("Best score (Neg. MAE): ", rf_grid_search.best_score_)

Best parameters found:  {'max_depth': 40, 'max_features': 0.2, 'n_estimators': 600}
Best score (Neg. MAE):  -31.240098586105024


In [10]:
rf6 = RandomForestRegressor(n_estimators = 600, min_samples_split = 2, min_samples_leaf = 1, max_features = 0.2, max_depth = 40, 
                            random_state = 123).fit(X_train, y_train)
y_pred = rf6.predict(X_test)
results_rf6 = pd.DataFrame({"id": id_test, "Tm": y_pred})
results_rf6.index.name = "id"
results_rf6.to_csv("Submissions/prediction_rf6.csv", index=False)

### 2.) SVR with radial kernel

In [11]:
parameters_grid = {
    "C": [0.1, 1, 10, 100, 1000],
    "epsilon": [0.01, 0.1, 0.5, 1],
    "gamma": ["scale", "auto", 0.001, 0.01, 0.1]}

svr_rbf = SVR(kernel = "rbf")

svr_rbf_grid_search = GridSearchCV(
    estimator = svr_rbf,
    param_grid = parameters_grid,
    cv = 4,
    n_jobs = -1,
    scoring = "neg_mean_absolute_error",
    error_score = "raise")

svr_rbf_grid_search.fit(X_train, y_train)

print("Best parameters found: ", svr_rbf_grid_search.best_params_)
print("Best score (Neg. MAE): ", svr_rbf_grid_search.best_score_)

Best parameters found:  {'C': 1000, 'epsilon': 1, 'gamma': 'auto'}
Best score (Neg. MAE):  -35.10910023103887


In [12]:
svr_rbf_4 = SVR(kernel = "rbf", gamma = "auto", epsilon = 1, C = 1000).fit(X_train, y_train)

y_pred = svr_rbf_4.predict(X_test)
results_svr_rbf_4 = pd.DataFrame({"id": id_test, "Tm": y_pred})
results_svr_rbf_4.index.name = "id"
results_svr_rbf_4.to_csv("Submissions/prediction_svr_rbf_4.csv", index=False)

### 3.) SVR with polynomial kernel

In [14]:
parameters_grid = {
    "C": [1, 10, 100],
    "epsilon": [0.01, 0.1],
    "degree": [2, 3]} 
    # reduced grid cause it took veeery long when I tried it with a bigger one

svr_poly = SVR(kernel = "poly", gamma = "scale", coef0 = 0)

svr_poly_grid_search = GridSearchCV(
    estimator = svr_poly,
    param_grid = parameters_grid,
    cv = 4,
    n_jobs = -1,
    scoring = "neg_mean_absolute_error",
    error_score = "raise")

svr_poly_grid_search.fit(X_train, y_train)

print("Best parameters found: ", svr_poly_grid_search.best_params_)
print("Best score (Neg. MAE): ", svr_poly_grid_search.best_score_)

Best parameters found:  {'C': 100, 'degree': 2, 'epsilon': 0.1}
Best score (Neg. MAE):  -53.948772079692006


In [15]:
svr_poly_2 = SVR(kernel = "poly", gamma = "scale", epsilon = 0.1, C = 100, degree = 2, coef0 = 0).fit(X_train, y_train)

y_pred = svr_poly_2.predict(X_test)
results_svr_poly_2 = pd.DataFrame({"id": id_test, "Tm": y_pred})
results_svr_poly_2.index.name = "id"
results_svr_poly_2.to_csv("Submissions/prediction_svr_poly_2.csv", index=False)